In [0]:
# Recurrent Neural Network

# Importing a data -> Selecting data/cols for training
# training data feature scaling
# Creata a list with each index having list of 60 values and o/p with one value (X_train, y_train)
# Convert them into a array
# Reshaping them into format required by RNN

# building a RNN network
# initializing RNN
# adding a hidden layers
# Compiling a RNN
# fitting RNN for training on train set

# import the test_data set -> selecting a req. column from a main dataframe
# creating a 'input' variable from df which will take last 60 values from training set from which it will predict first value of o/p
# reshape and transform the variable

# prepare the x_test with last 60 value of x_train 
# np.array, reshape 
# using .predict for predicting 


# Part 1 - Importing libararies and data files

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [32]:
# file path - /content/1.csv
import glob

path = r'/content/' # use your path
all_files = glob.glob(path + "/*.csv")
all_files = list(all_files)
print(all_files[0:3])
#file_names = all_files.replace('/content/','')
file_name = []
for y in all_files:
  file_list = (y.strip('/content/'))
  file_name.append(file_list)

print(file_name)

#print(file_name[0],file_name[1],file_name[2],file_name[3])

['/content/81.csv', '/content/80.csv']
['81.csv', '80.csv']


In [0]:
def file_processing(filename):  
  
  print(' Importing the training set :\n')
  print('file getting processed inside function:',filename)

  df = pd.read_csv(filename)

  row_count = round((len(df)-30))
  print('row_count :', row_count)

  dataset_train = df.iloc[:row_count, 4:5]
  training_set = dataset_train.values

  # n_steps = 60
  n_features = 1

  # print(training_set.dtype)
  # print(training_set[0:5])
  print('len(df)',len(df))
  print('len(training_set)',len(training_set))
  # print(len(dataset_train))

  print('######  Feature Scaling : ######\n')

  from sklearn.preprocessing import MinMaxScaler
  sc = MinMaxScaler(feature_range = (0, 1))
  training_set_scaled = sc.fit_transform(training_set)
  print(len(training_set_scaled))

  # Creating a data structure with 60 timesteps and 1 output
  print('######  Creating a data structure with 60 timesteps and 1 output : ######\n')

  X_train = []
  y_train = []
  for i in range(60, len(training_set_scaled)):
      X_train.append(training_set_scaled[i-60:i, 0])
      y_train.append(training_set_scaled[i, 0])
  X_train, y_train = np.array(X_train), np.array(y_train)

  # Reshaping
  print('######  Reshaping : ######\n')

  X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

  # print('X_train.shape nparray:', X_train.shape)
  # print('(X_train[0:1] np.array :\n', len(X_train[0:1]))
  # print('X_train.shape[0] :', X_train.shape[0])
  # print('X_train.shape[1] :', X_train.shape[1])
  # print('X_train.shape[2] :', X_train.shape[2])

  # Part 2 - Building the RNN
  print('###### Part 2 - Building the RNN : ###### \n')

  # Importing the Keras libraries and packages
  from keras.models import Sequential
  from keras.layers import Dense
  from keras.layers import LSTM
  from keras.layers import Dropout

  # Initialising the RNN
  model = Sequential()

  # Stacked LSTM
  # Multiple hidden LSTM layers can be stacked one on top of another in what is referred to as a Stacked LSTM model.
  # An LSTM layer requires a three-dimensional input and LSTMs by default will produce a two-dimensional output as an interpretation from the end of the sequence.
  # We can address this by having the LSTM output a value for each time step in the input data by setting the return_sequences=True argument on the layer. This allows us to have 3D output from hidden LSTM layer as input to the next.

  # Adding the first LSTM layer and some Dropout regularisation
  model.add(LSTM(units = 200, return_sequences = True, input_shape = (X_train.shape[1], n_features),activation='relu')) # input_shape=(n_steps, n_features)
  model.add(Dropout(0.2))

  # Adding a second LSTM layer and some Dropout regularisation
  model.add(LSTM(units = 200, return_sequences = True)) # activation='relu' at each LSTM layers
  model.add(Dropout(0.2))

  # Adding a third LSTM layer and some Dropout regularisation
  model.add(LSTM(units = 200, return_sequences = True))
  model.add(Dropout(0.2))

  # Adding a Extra LSTM layer and some Dropout regularisation
  model.add(LSTM(units = 100, return_sequences = True))
  model.add(Dropout(0.2))


  # Adding a fourth LSTM layer and some Dropout regularisation
  model.add(LSTM(units = 100))
  model.add(Dropout(0.2))

  # Adding the output layer
  model.add(Dense(units = 1,activation='relu'))

  # Compiling the RNN
  #model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
  model.compile(optimizer = 'adam', loss = 'mean_squared_error',metrics= ['mean_squared_error'])


  # Fitting the RNN to the Training set
  model.fit(X_train, y_train, epochs = 100, batch_size = 512)

  # Part 3 - Making the predictions and visualising the results
  print('####### Part 3 - Making the predictions and visualising the results ####### \n')
  # Getting the real stock price of test data
  #dataset_test = pd.read_csv('Test_data_1.csv')
  dataset_test = df.iloc[row_count:,4:5]
  real_stock_price = dataset_test.values

  # Getting the predicted stock price of test data

  print('####### Getting the predicted stock price of test data ###### \n')
  dataset_total = pd.concat((dataset_train['close'], dataset_test['close']), axis = 0)
  inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
  print('inputs values', len(inputs))
  print('inputs shape', inputs.shape)

  inputs = inputs.reshape(-1,1)
  # print('inputs reshape', len(inputs))
  print('inputs reshape', inputs.shape)
  # print('dataset_total :', len(dataset_total))

  inputs = sc.transform(inputs)
  # print('inputs transform', len(inputs))
  print('inputs transform', inputs.shape)


  print('####### Model Prediction ####### \n')
  X_test = []
  for i in range(60, len(dataset_test)+60):
      X_test.append(inputs[i-60:i, 0])

  #print('(X_test[0:3] :\n', X_test[0:3])
  print('len of X_test', len(X_test))
  print('len of [X_test 0]:', len(X_test[0]))

  print('(X_test[0:1] :\n', X_test[0:1])

  X_test = np.array(X_test)
  # print('X_test.shape nparray:', X_test.shape)
  # print('(X_test[0:1] np.array :\n', X_test[0:1])

  # print('X_test.shape[0] :', X_test.shape[0])
  # print('X_test.shape[1] :', X_test.shape[1])
  # print('X_test.shape[1] :', X_test.shape[2])

  # The model expects the input shape to be three-dimensional with [samples, timesteps, features]
  X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
  print('X_test reshape:', len(X_test))

  predicted_stock_price = model.predict(X_test)
  predicted_stock_price = sc.inverse_transform(predicted_stock_price)


  # Visualising the results
  print('###### Visualising the results ######\n')

  plt.plot(real_stock_price, color = 'red', label = 'Real Stock Price')
  plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted  Stock Price')
  plt.title('Stock Price Prediction')
  plt.xlabel('Time')
  plt.ylabel('Stock Price')
  plt.legend()
  mse_chart =  (f'{filename}_MeanSquareError.png')
  print('Name of .png file mse_chart', mse_chart)
  plt.savefig(mse_chart)
  plt.show()

  ## Model summary
  print('dataset_train.info()', dataset_train.info())
  print('dataset_test.info()', dataset_test.info())
  model.summary()
  #print('model_list,model_summary', model_list,model_summary)
  #return model_list,model_summary



In [0]:
for i in range(len(file_name)):
  #filename = all_files[i]
  csv_file = file_name[i]
  print('file getting processed :',file_name[i])
  #model_list,model_summary = file_processing(file_name[i])
  file_processing(file_name[i])

file getting processed : 81.csv
 Importing the training set :

file getting processed inside function: 81.csv
row_count : 5620
len(df) 5650
len(training_set) 5620
######  Feature Scaling : ######

5620
######  Creating a data structure with 60 timesteps and 1 output : ######

######  Reshaping : ######

###### Part 2 - Building the RNN : ###### 

Epoch 1/100
5560/5560 [==============================] - 78s 14ms/step - loss: 0.0262 - mean_squared_error: 0.0262
Epoch 2/100
5560/5560 [==============================] - 64s 11ms/step - loss: 0.0051 - mean_squared_error: 0.0051
Epoch 3/100
5560/5560 [==============================] - 64s 12ms/step - loss: 0.0029 - mean_squared_error: 0.0029
Epoch 4/100
5560/5560 [==============================] - 64s 12ms/step - loss: 0.0021 - mean_squared_error: 0.0021
Epoch 5/100
5560/5560 [==============================] - 63s 11ms/step - loss: 0.0017 - mean_squared_error: 0.0017
Epoch 6/100
5560/5560 [==============================] - 63s 11ms/step - los

In [0]:
#  for l in ([1,2,31,67]):
#   mse_chart =  (f'{l}_MeanSquareError.png')
#   print('Name of .png file mse_chart', mse_chart)
#   #plt.savefig(mse_chart)
#   #plt.show()

In [0]:
# print(len(predicted_stock_price))
# print(predicted_stock_price[0:5])

In [0]:
# len(dataset_test)

In [0]:
# df['Predicted Price'] = np.NAN
# print(df['Predicted Price'].shape)
# df.iloc[row_count:,6] = predicted_stock_price



In [0]:
# df.to_csv('Predicted stock price.csv')
# #print(df.head(-1))

In [0]:

#model.summary()

In [0]:
# # Step 8: Evaluate model
# scores = model.evaluate(X_test, predicted_stock_price)
# print("Error: %.2f%%" % (100-scores[1]*100))

In [0]:
# import numpy as np
# y = np.zeros((2,3,4))
# print(y)

In [0]:
# print(y.shape[0])

In [0]:
# y = y.reshape(8,3)
# print(y)